# [75.06 / 95.58] Organización de Datos <br> Trabajo Práctico 2: Machine Learning
# Notebook Principal

**Grupo 30: Datatouille**

- 101055 - Bojman, Camila
- 100029 - del Mazo, Federico
- 100687 - Hortas, Cecilia
- 97649 - Souto, Rodrigo

**http://fdelmazo.github.io/7506-Datos/**

**https://www.kaggle.com/datatouille2018/competitions**

Continuando la investigación sobre la empresa Trocafone realizada en el [TP1](https://fdelmazo.github.io/7506-Datos/TP1/TP1.html), se busca determinar la probabilidad de que un usuario del sitio realice una conversión en el período determinado.

Notebooks en orden de corrida y lectura:

0. [TP1](https://fdelmazo.github.io/7506-Datos/TP1/TP1.html) --> Familiarización con el set de datos y exploración de estos.

1. [Investigación Previa](https://fdelmazo.github.io/7506-Datos/TP2/investigacion.html) --> Con ayuda de lo trabajado en el TP1, se averiguan más cosas de las datos, en busqueda de que poder reutilizar.

2. [Creación de Dataframes](https://fdelmazo.github.io/7506-Datos/TP2/new-dataframes.html) --> Como parte del feature engineering, se crean dataframes nuevos con información de los productos del sitio y de como se accede a este (marcas, sistemas operativos, etc).

3. [Feature Engineering](https://fdelmazo.github.io/7506-Datos/TP2/feature-engineering.html) --> Busqueda de atributos de los usuarios de los cuales se busca predecir la conversión.

4. [Submission Framework](https://fdelmazo.github.io/7506-Datos/TP2/submission-framework.html) --> Pequeño framework para construir las postulaciones de labels. 

5. TP2 (este notebook)--> Teniendo todo en cuenta, usando los dataframes con todos los atributos buscados y encontrados, se definen y aplican los algoritmos de clasificación, se realizan los entrenamientos y posteriores predicciones de conversiones y finalmente se arman las postulaciones de labels.

In [ ]:
# Set-up inicial, se deja comentado para evitar instalarle módulos al usuario
## Primero, descargar los datasets de no tenerlos

# Antes de comenzar, setear las credenciales (usuario y token)

# 1. Visitar: https://www.kaggle.com/datatouille2018/account (con la cuenta que sea)
# 2. Tocar en Create New API Token
# 3. Guardar el archivo descargado en ~/.kaggle/kaggle.json

# !pip install kaggle # https://github.com/Kaggle/kaggle-api
# !kaggle competitions download -c trocafone -p data
# !unzip -q data/events_up_to_01062018.csv.zip -d data
# !rm data/events_up_to_01062018.csv.zip
# !ls data/

## Luego, descargar los módulos a utilizar a lo largo de todo el trabajo

# !pip install nbimporter
# !conda install -c conda-forge xgboost 

In [ ]:
import nbimporter # pip install nbimporter
import pandas as pd
import numpy as np
import calendar
import submission_framework as SF

In [ ]:
df_users = pd.read_csv('data/user-features.csv',low_memory=False).set_index('person')
df_y = pd.read_csv('data/labels_training_set.csv').groupby('person').sum()

display(df_users.head(), df_y.head())

## Algoritmos de Machine Learning

In [ ]:
seed = 42
test_size = 0.34

def full_framework_wrapper(model_name, model_function, columns=df_users.columns.tolist(), test_size=test_size, seed=seed, verbosity=False, all_in=False, submit=False):
    model_df_x = df_users[columns]
    model_df_y = df_y
    
    X, y = SF.fr1_extract_X_y(model_df_x, model_df_y)
    X_train, X_test, y_train, y_test = SF.fr2_train_test_split(X, y, seed, test_size)
    if all_in:
        model = model_function(X, y, seed)
    else:
        model = model_function(X_train, y_train, seed)
    accuracy = SF.fr4_accuracy_score(X_test, y_test, model, model_name)
    auc = SF.fr5_auc(X_test, y_test, model, model_name)
    X_to_predict, predictions = SF.fr6_extract_X_to_predict(model_df_x, model_df_y, model)
    
    if verbosity: 
        feature_importances = SF.fr7_print_information(model_df_x, model, X_to_predict, True)
        display(feature_importances)
    
    display('Model: {} - Accuracy: {:.4f} - AUC: {:.4f}'.format(model_name, accuracy, auc))  
    
    if submit:
        csv_name, submission = SF.fr8_to_csv(X_to_predict, predictions, model_name, auc)
        display(csv_name)
        return auc, csv_name
    
    return auc

---

### Decision Tree


In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz

def decision_tree(X_train, y_train, seed):
    tree = DecisionTreeClassifier(criterion='gini',
                              min_samples_leaf=5,
                              min_samples_split=5,
                              max_depth=3,
                              random_state=seed)

    tree.fit(X_train, y_train)
    return tree

full_framework_wrapper('decision_tree',decision_tree)

---

### Random Forest

In [ ]:
ss_cols_1 = ['total_checkouts_month_5',
'timestamp_last_checkout',
'timestamp_last_event',
'has_checkout_month_5',
'total_checkouts',
'days_to_last_event',
'total_checkouts_last_week',
'total_checkouts_months_1_to_4',
'total_conversions',
'total_session_conversions',
'total_events',
'total_sessions',
'avg_events_per_session',
'total_session_checkouts',
'has_checkout']

from sklearn.ensemble import RandomForestClassifier

def random_forest(X_train, y_train, seed):
    rf = RandomForestClassifier(n_estimators=20,
                           n_jobs=2,
                           min_samples_split=300,
                           random_state=seed,
                           class_weight='balanced')

    y_train.shape = y_train.shape[0]
    rf.fit(X_train, y_train)
    return rf

full_framework_wrapper('random_forest',random_forest,columns=ss_cols_1)

In [ ]:
# NO DAR BOLA POR AHORA

# No era necesario hacer el entrenamiento final con el modelo
# entrenado con X_train. Conviene usar el set de entrenamiento
# entero.

# Lo de X_train y X_test es sólo (POR AHORA) para mostrar
# el accuracy_score choto que estamos teniendo

full_framework_wrapper('random_forest_allin',random_forest,all_in=True,submit=True)


---

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model_name = 'knn'

rango = 5
def knn_gridsearch(x_train, y_train, x_test, y_test):
    y_train.shape = y_train.shape[0]
    k_max = (0,0)
    for k in range(1,100, rango):
        knn = KNeighborsClassifier(n_neighbors=k, weights='uniform', n_jobs=-1)
        knn.fit(x_train, y_train)
        score = knn.score(x_test, y_test)*100
        if score > k_max[1]:
            k_max = (k,score)
        print("K: {}, {}".format(k, score))
    a = k_max[0]
    k_max = (0,0)
    for k in range(a - rango, a + rango):
        knn = KNeighborsClassifier(n_neighbors=k, weights='uniform', n_jobs=-1)
        knn.fit(x_train, y_train)
        score = knn.score(x_test, y_test)*100
        if score > k_max[1]:
            k_max = (k,score)
        print("K: {}, {}".format(k, score))
    return k_max[0]

K = knn_gridsearch(X_train, y_train, X_test, y_test)

def knn(X_train, y_train, seed, k=K):
    knn = KNeighborsClassifier(n_neighbors=k, weights='uniform', n_jobs=-1)
    y_train.shape = y_train.shape[0]
    knn.fit(X_train, y_train)
    return knn

full_framework_wrapper(f'KNN{K}',knn)

---

### XGBoost


In [ ]:
import xgboost as xgb #conda install -c conda-forge xgboost 

def xgboost(X_train, y_train, seed):
    xg_reg = xgb.XGBClassifier(objective='reg:linear',
                class_weight='balanced',
                colsample_bytree=0.3, learning_rate=0.1,
                min_child_weight=2,
                max_depth=5, alpha=10, n_estimators=20,
                random_state=seed)
    y_train.shape = y_train.shape[0]
    xg_reg.fit(X_train,y_train)
    return xg_reg

full_framework_wrapper('xgboost',xgboost,columns=ss_cols_1)

In [ ]:
# NO BORRAR ESTO !!!!

total_predictions = 0
tmp_seed = seed
for i in range(50):
    tmp_seed = tmp_seed + i
    model = xgboost(X, y, tmp_seed)
    print(tmp_seed)
    X_to_predict, predictions = SF.fr6_extract_X_to_predict(fr_df, fr_df_y, model)
    total_predictions = total_predictions + predictions

predictions = total_predictions / 50

"""
model = xgboost(X, y, seed)
X_to_predict, predictions = SF.fr6_extract_X_to_predict(fr_df, fr_df_y, model)
"""

In [ ]:
for i in range(len(predictions)):
    if predictions[i] < 0:
        print("WARNING: prediction[{}] = {}".format(i, predictions[i]))
        predictions[i] = 0

---

## Encontrando el mejor algoritmo y sus mejores hiperparametros

1. Usando Random Forest encontramos cuanto aporta cada feature. Ordenamos en orden de importancia

2. Generamos una lista de listas acumulativas de los features

3. Corremos todos los algoritmos, entrenando con X_train, sobre todas las listas, en busqueda del mayor AUC

4. Una vez encontrado el mejor algoritmo, se submitea entrenando con todo X

In [ ]:
X, y = SF.fr1_extract_X_y(df_users, df_y)
X_train, X_test, y_train, y_test = SF.fr2_train_test_split(X, y, seed, test_size)
model = random_forest(X_train, y_train, seed)
X_to_predict, predictions = SF.fr6_extract_X_to_predict(df_users, df_y, model)
feature_importances = SF.fr7_print_information(df_users, model, X_to_predict, True)

display(feature_importances)

In [ ]:
features_ordenados = feature_importances.index.tolist()
lista_progresiva_de_cols = [features_ordenados[:i] for i in range(1,len(features_ordenados))]

In [ ]:
posibilidades = [('xgboost',xgboost),
                 ('random_forest',random_forest),
                 ('decision_tree',decision_tree),
                 (f'KNN{K}',knn)
                ]

max_auc = 0
campeon_indice = 0
campeon_nombre = ''
campeon_algoritmo = None

for i, cols in enumerate(lista_progresiva_de_cols):
    display(f'Iteracion {i} de {len(lista_progresiva_de_cols)}')
    for nombre,algoritmo in posibilidades:
        auc = full_framework_wrapper(nombre,algoritmo,columns=cols)
        if auc>max_auc:
            max_auc = auc
            campeon_indice = i
            campeon_nombre = nombre
            campeon_algoritmo = algoritmo
        
display(f"Mejor Apuesta: {campeon_nombre} ({max_auc:.2f} AUC), usando lista_progresiva_de_cols[{campeon_indice}]")

In [ ]:
display(lista_progresiva_de_cols[campeon_indice])

auc, csv_name = full_framework_wrapper(campeon_nombre+'_all_in',
                       campeon_algoritmo,
                       columns=lista_progresiva_de_cols[campeon_indice],
                       submit=True,
                       all_in=True)

In [ ]:
## Descomentar y submitear!
## Ojo, solo correr una vez!!!

%env FILE_NAME = {csv_name}
%env MESSAGE = Features: Lista acumulativa de {len(lista_progresiva_de_cols)} features ordenados por importancia, indice:{campeon_indice}.

# !kaggle competitions submit -f $FILE_NAME -m "$MESSAGE" trocafone

print('https://www.kaggle.com/c/trocafone/submissions?sortBy=publicScore&group=all&page=1&pageSize=20')
print('https://www.kaggle.com/c/trocafone/leaderboard')

---
---

## Lista de robos

TODO: Borrar

* [ ] https://github.com/urielkelman/abracadata/tree/master/TP2

* [ ] https://github.com/MatiasReimondo/Datos

* [ ] Ver diapos argerich feature engineering

* [ ] Spammear ramos mejia de preguntas

* [ ] Notebooks de Martin

* [ ] Notebook de Juan

## Lista de cosas a hacer

* [ ] Identificar bias y varianza, ploteando error de set de entrenamiento y error de set de test en funcion de cantidad de datos en set de entrenamiento (mismo plot)

* [ ] Perturbar datos de entrada -> reducir overfitting

* [ ] Plotear AUC nuestra, AUC kaggle segun submission

* [ ] Catboost

* [ ] Loopear interacciones importantes! Random forest claveee

* [ ] No perder nombre de feature original

* [ ] Scrapear trocafone --> Precios

* [ ] Xgboost no sirve para importancia de features. No es estable

* [ ] Reclamar 50 usd aws &#128544; 

* [ ] Clustering para nuevos features para entrenar